In [1]:
import numpy as np
import pandas as pd

## **DATA SELECTION**

First, we select the tables that contains the variables of interest

In [2]:
path1 = "../data/raw/The-Technical-Debt-Dataset-CSV/GIT_COMMITS.csv"
path2 = "../data/raw/The-Technical-Debt-Dataset-CSV/SONAR_ANALYSIS.csv"
path3 = "../data/raw/The-Technical-Debt-Dataset-CSV/SONAR_MEASURES.csv"
db_commits = pd.read_csv(path1)
db_analysis = pd.read_csv(path2)
db_measures = pd.read_csv(path3)

Secondly, for each table, we select those variables:

    · GIT_COMMITS: project id, commit hash, commit message, author and committer date
    · SONAR_ANALYSIS: revision and anlysis key
    · SONAR_MESURES: analysis key, complexity, violations and development cost 

In [3]:
db_commits = db_commits[["PROJECT_ID", "COMMIT_HASH", "COMMIT_MESSAGE", "AUTHOR",  "COMMITER_DATE"]]
db_commits.head()

,PROJECT_ID,COMMIT_HASH,COMMIT_MESSAGE,AUTHOR,COMMITER_DATE
0,org.apache:batik,52fc76012c5f969145c39d3fb398a7c2c094474f,New repository initialized by cvs2svn.\n\ngit-...,No Author,2000-10-01T07:37:01Z
1,org.apache:batik,b1ff4af6abfec32fc710d77795bb20a612a82126,Initial revision\n\n\ngit-svn-id: https://svn....,James Duncan Davidson,2000-10-01T07:37:01Z
2,org.apache:batik,c8d7a13470987f892f7466d55c10e3cee34de31d,Update\nPR:\n\n\ngit-svn-id: https://svn.apach...,James Duncan Davidson,2000-10-01T07:40:39Z
3,org.apache:batik,93a16402b48ae1cf70ea4bd030479170749bd10a,Added question line (more a test of list/forwa...,James Duncan Davidson,2000-10-01T08:15:04Z
4,org.apache:batik,fcaecb541edc03f36b6ec7a792e97dfeabf26117,testing commit\n\n\ngit-svn-id: https://svn.ap...,Dean Jackson,2000-10-02T13:33:11Z


In [4]:
db_analysis = db_analysis[["REVISION", "ANALYSIS_KEY"]]
db_analysis.head()

,REVISION,ANALYSIS_KEY
0,b9988a83e364b9b470873dff8996dcf401d08dc4,AWedEXD3C4KKKThcCqHV
1,5d351ea375f2b8d4ca0ae40b887392915983121a,AWedCu_QC4KKKThcCqGv
2,2bb54cc863c9c599e316ae5da7ba1a77f42e9402,AWedBDbIC4KKKThcCqGA
3,68b851d2adbbeee6014d1c37b9c07fdb4fc6820f,AWec_b8nC4KKKThcCqFg
4,e6c057bd6a04ec286d4ac4da5d2a82696868b937,AWec904qC4KKKThcCqE3


In [5]:
db_measures = db_measures[["analysis_key", "complexity", "violations", "development_cost"]]
db_measures.head()

,analysis_key,complexity,violations,development_cost
0,AWedEXD3C4KKKThcCqHV,30703.0,16006,6207000
1,AWedCu_QC4KKKThcCqGv,30703.0,16006,6207000
2,AWedBDbIC4KKKThcCqGA,30703.0,16004,6207330
3,AWec_b8nC4KKKThcCqFg,30699.0,16000,6206280
4,AWec904qC4KKKThcCqE3,30670.0,15991,6200220


## **DATA CLEANING**

We proceed to clean the tables by deleting the NA only in the sonar_measures table as we want to keep the time consistency when computing the invervals between the selected variables.

In [6]:
db_measures = db_measures[db_measures["complexity"].notna()]

In [17]:
db_sonar0 = db_analysis.merge(db_measures, left_on="ANALYSIS_KEY", right_on="analysis_key")
db_sonar = db_sonar0.rename(columns = {'REVISION' : 'COMMIT_HASH'})
db_sonar = db_sonar[["COMMIT_HASH", "complexity", "violations", "development_cost"]]
db_sonar.head()

,COMMIT_HASH,complexity,violations,development_cost
0,b9988a83e364b9b470873dff8996dcf401d08dc4,30703.0,16006,6207000
1,5d351ea375f2b8d4ca0ae40b887392915983121a,30703.0,16006,6207000
2,2bb54cc863c9c599e316ae5da7ba1a77f42e9402,30703.0,16004,6207330
3,68b851d2adbbeee6014d1c37b9c07fdb4fc6820f,30699.0,16000,6206280
4,e6c057bd6a04ec286d4ac4da5d2a82696868b937,30670.0,15991,6200220


In [19]:
db_commits

,PROJECT_ID,COMMIT_HASH,COMMIT_MESSAGE,AUTHOR,COMMITER_DATE
0,org.apache:batik,52fc76012c5f969145c39d3fb398a7c2c094474f,New repository initialized by cvs2svn.\n\ngit-...,No Author,2000-10-01T07:37:01Z
1,org.apache:batik,b1ff4af6abfec32fc710d77795bb20a612a82126,Initial revision\n\n\ngit-svn-id: https://svn....,James Duncan Davidson,2000-10-01T07:37:01Z
2,org.apache:batik,c8d7a13470987f892f7466d55c10e3cee34de31d,Update\nPR:\n\n\ngit-svn-id: https://svn.apach...,James Duncan Davidson,2000-10-01T07:40:39Z
3,org.apache:batik,93a16402b48ae1cf70ea4bd030479170749bd10a,Added question line (more a test of list/forwa...,James Duncan Davidson,2000-10-01T08:15:04Z
4,org.apache:batik,fcaecb541edc03f36b6ec7a792e97dfeabf26117,testing commit\n\n\ngit-svn-id: https://svn.ap...,Dean Jackson,2000-10-02T13:33:11Z
...,...,...,...,...,...
81067,org.apache:thrift,53d9c0c20bd5af65676928b9b7a73dcb2cad3d78,Merging EOFException changes from Ben Maurer ...,Mark Slee,2007-11-26 21:15:40+00:00
81068,org.apache:thrift,5ab570558f55d73472fbf6c0e66e6e165093c7d8,Fix writeContainerEnd call being inside loop i...,Mark Slee,2007-11-27 08:38:16+00:00
81069,org.apache:thrift,844ac12489600d7647f01ab4f9b99d9e1b81e69e,TJSONProtocol writing support in Java Summary...,Mark Slee,2007-11-27 08:38:52+00:00
81070,org.apache:thrift,256bdc444866b90bbdccfb5343e9c9ea8c22603c,IPv6 tweaks for Thrift Summary: Need to pass ...,Mark Slee,2007-11-27 08:42:19+00:00


In [33]:
db_merged = db_commits.merge(db_sonar, how = 'left', left_on = 'COMMIT_HASH', right_on = 'COMMIT_HASH', indicator = True)
db_mergeda = db_commits.merge(db_sonar, how = 'inner', left_on = 'COMMIT_HASH', right_on = 'COMMIT_HASH')
print(db_commits.size)
db_merged = db_merged[db_merged['_merge'] == 'left_only']
print(db_merged.size)

405360
366543
